- 다중분류 문제
  - 정답 데이터
    - 1차원 형태 텐서, dtype: int64
  - 예측값 확률 변경 시
    - softmax()함수 사용
  - 예측 확률 class번호 결정
    - np.argmax()
  - 손실함수
    - CrossEntrophy()

# 데이터 불러오기

In [1]:
!gdown 1Jy5vgN8LvHTMpXkJeR0hO_b74VAvt8K_
!unzip -oqq meat.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Jy5vgN8LvHTMpXkJeR0hO_b74VAvt8K_
From (redirected): https://drive.google.com/uc?id=1Jy5vgN8LvHTMpXkJeR0hO_b74VAvt8K_&confirm=t&uuid=2065fd49-451c-44db-9ca8-6ce65731929b
To: /content/meat.zip
100% 61.8M/61.8M [00:01<00:00, 41.7MB/s]


# 재현성 구현

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
SEED = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
train = pd.read_csv('train/class_info.csv')
test = pd.read_csv('test/class_info.csv')

train.shape, test.shape

((1246, 2), (1020, 2))

In [5]:
train_path = ('train/' + train['filename']).to_numpy()
test_path = ('test/' + test['filename']).to_numpy()

train_path

array(['train/1074.jpg', 'train/1222.jpg', 'train/2105.jpg', ...,
       'train/776.jpg', 'train/750.jpg', 'train/948.jpg'], dtype=object)

- 정답 데이터만들기
  - 0: 신선한 고기
  - 1: 먹을만한 고기
  - 2: 상한 고기

In [6]:
target = train['target'].to_numpy()
target.shape, target.dtype

((1246,), dtype('int64'))

# Compose 클래스 구현, transform 객체

In [7]:
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2 # pillow객체 못받음, numpy배열로 받아야됨

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [8]:
resize = [224,224]
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_lst = [
    A.Resize(*resize), # 모든 이미지 크기 통일화
    A.Normalize(mean= mean, std= std), # 정규화
    ToTensorV2()
]

train_trans = A.Compose(train_lst)

test_lst = [
    A.Resize(*resize),
    A.Normalize(mean= mean, std= std),
    ToTensorV2()
]

test_trans = A.Compose(test_lst)

# 데이터셋 클래스구현

In [9]:
class MeatDataset(torch.utils.data.Dataset):
  def __init__(self,trans,x,y=None):
    self.trans = trans
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    item = {}
    x = cv2.imread(self.x[idx]) # BGR
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB) # RGB
    item['x'] = self.trans(image = x)['image']

    if self.y is not None:
      item['y'] = torch.tensor(self.y[idx])

    return item

- 결과 확인하기

In [10]:
dt = MeatDataset(train_trans, train_path, target)
dt[0]

{'x': tensor([[[ 1.5468,  1.4269,  1.5639,  ...,  1.4269,  0.8618,  0.6049],
          [ 1.4612,  1.4440,  1.4954,  ...,  1.5982,  1.3755,  0.9132],
          [ 1.4269,  1.4783,  1.4269,  ...,  1.6838,  1.5810,  1.3927],
          ...,
          [ 0.4166,  0.5193,  0.5536,  ...,  0.2282,  0.1768,  0.3994],
          [ 0.4508,  0.5707,  0.5878,  ..., -0.0801,  0.1083,  0.2453],
          [ 0.4337,  0.5536,  0.5707,  ...,  0.0398, -0.1143,  0.2624]],
 
         [[ 1.5007,  1.3782,  1.5182,  ...,  1.3431,  0.6779,  0.3803],
          [ 1.4132,  1.3957,  1.4482,  ...,  1.5357,  1.2206,  0.6954],
          [ 1.3782,  1.4307,  1.3782,  ...,  1.6232,  1.4482,  1.2206],
          ...,
          [ 0.2402,  0.3452,  0.3803,  ..., -0.5826, -0.6352, -0.4076],
          [ 0.2927,  0.3978,  0.4153,  ..., -0.8978, -0.7402, -0.6001],
          [ 0.2577,  0.3803,  0.3978,  ..., -0.8102, -0.9678, -0.5826]],
 
         [[ 1.5071,  1.3851,  1.5245,  ...,  1.4025,  0.7751,  0.5136],
          [ 1.4200,  1.

In [11]:
dl = torch.utils.data.DataLoader(dt, batch_size = 2, shuffle= False)
batch = next(iter(dl))
batch

{'x': tensor([[[[ 1.5468,  1.4269,  1.5639,  ...,  1.4269,  0.8618,  0.6049],
           [ 1.4612,  1.4440,  1.4954,  ...,  1.5982,  1.3755,  0.9132],
           [ 1.4269,  1.4783,  1.4269,  ...,  1.6838,  1.5810,  1.3927],
           ...,
           [ 0.4166,  0.5193,  0.5536,  ...,  0.2282,  0.1768,  0.3994],
           [ 0.4508,  0.5707,  0.5878,  ..., -0.0801,  0.1083,  0.2453],
           [ 0.4337,  0.5536,  0.5707,  ...,  0.0398, -0.1143,  0.2624]],
 
          [[ 1.5007,  1.3782,  1.5182,  ...,  1.3431,  0.6779,  0.3803],
           [ 1.4132,  1.3957,  1.4482,  ...,  1.5357,  1.2206,  0.6954],
           [ 1.3782,  1.4307,  1.3782,  ...,  1.6232,  1.4482,  1.2206],
           ...,
           [ 0.2402,  0.3452,  0.3803,  ..., -0.5826, -0.6352, -0.4076],
           [ 0.2927,  0.3978,  0.4153,  ..., -0.8978, -0.7402, -0.6001],
           [ 0.2577,  0.3803,  0.3978,  ..., -0.8102, -0.9678, -0.5826]],
 
          [[ 1.5071,  1.3851,  1.5245,  ...,  1.4025,  0.7751,  0.5136],
        

# 신경망 클래스 구현

## timm 라이브러리
- 이미지 분류의 다양한 사전학습 모델 제공
- pytorch 기반, 다양한 이미지 관련 모델의 아키텍처를 사용할 수 있는 라이브러리

In [12]:
import timm

In [13]:
model_name = 'efficientvit_b1.r224_in1k'
model_name

'efficientvit_b1.r224_in1k'

## create_model 함수
- 사전학습된 모델을 pytorch모델 객체로 반환
- 첫 번째 인수: model ID명
- 두 번째 인수: pretrained
- 세 번째 인수: num_class
  - 이진분류 문제: num_classes= 1
  - 다중분류 문제: num_classes= 3

In [14]:
# model = timm.create_model(model_name, pretrained= True, num_classes= 3).to(device) # 이진분류 문제 > num_classes=1, 다중분류 문제 > num_classes= 3
# model

In [15]:
class Net(torch.nn.Module):
  def __init__(self, model_name, num_classes):
    super().__init__()
    self.pre_model = timm.create_model(model_name= model_name, pretrained= True, num_classes= num_classes).to(device)

  def forward(self,x):
    return self.pre_model(x)

In [16]:
model = Net(model_name, 3)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

Net(
  (pre_model): EfficientVit(
    (stem): Stem(
      (in_conv): ConvNormAct(
        (dropout): Dropout(p=0.0, inplace=False)
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): Hardswish()
      )
      (res0): ResidualBlock(
        (pre_norm): Identity()
        (main): DSConv(
          (depth_conv): ConvNormAct(
            (dropout): Dropout(p=0.0, inplace=False)
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (norm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): Hardswish()
          )
          (point_conv): ConvNormAct(
            (dropout): Dropout(p=0.0, inplace=False)
            (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm): BatchNorm2d(16, eps=1e-05, 

# 학습데이터 loop함수 구현

In [17]:
def train_loop(dl, model, loss_fn, optimizer, device):
  epoch_loss = 0
  model.train() # 학습모드

  for batch in dl:
    pred = model(batch['x'].to(device)) # gpu에서 예측값 계산
    loss = loss_fn(pred, batch['y'].to(device)) # gpu에서 손실함수로 예측값, 정답데이터

    optimizer.zero_grad() # 기울기 0초기화
    loss.backward() # 역전파
    optimizer.step() # 가중치업데이트

    epoch_loss += loss.item() # 손실값> 파이썬데이터유형으로 변형해 저장
  epoch_loss /= len(dl) # epoch 평균 손실계산
  return epoch_loss

# 테스트데이터 loop함수 구현

In [18]:
@torch.no_grad() # 기울기 0으로 초기화 실행 x
def test_loop(dl, model, loss_fn, device): # optimizer 제외하기
  epoch_loss = 0
  model.eval() # 평가모드

  act = torch.nn.Softmax(dim = 1) # Softmax활성화함수 > 다중분류에서만 가능
  pred_list = []

  for batch in dl:
    pred = model(batch['x'].to(device)) # gpu에서 예측값 계산
    if batch.get('y') is not None: # y값이 있을 경우
      loss = loss_fn(pred, batch['y'].to(device)) # gpu에서 손실함수로 예측값, 정답데이터
      epoch_loss += loss.item() # 손실값> 파이썬데이터유형으로 변형해 저장

    pred = act(pred) # 활성화함수 적용 > 회귀에서 사용 x
    pred = pred.to('cpu').numpy() # gpu에 있는 예측값 cpu로 전환 후 numpy로 저장
    pred_list.append(pred)

  pred = np.concatenate(pred_list) # 예측값 합치기
  epoch_loss /= len(dl) # epoch 평균 손실계산
  return epoch_loss, pred # epoch_loss, pred 반환

# 하이퍼파라미터 정의

In [19]:
batch_size = 32 # 배치사이즈 설정
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류문제이므로 CrossEntropy
epochs = 100 # epoch 횟수
n_splits = 5 #kfold의 k값

# 조합 후 KFold학습 수행

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
cv = KFold(n_splits, random_state = SEED, shuffle = True)

In [21]:
is_holdout = False # False: onehot 실행x, True: onehot 실행o
reset_seeds(SEED) # 시드고정
score_list = []

for i, (tri,vai) in enumerate(cv.split(train_path, target)): # enumerate로 인덱스+tri,vai
  model = Net(model_name, 3).to(device) # 신경망 모델 gpu or cpu 전환
  optimizer = torch.optim.Adam(model.parameters()) # 최적화> Adam, model설정 후 파라미터 함수실행

  # 학습데이터
  train_dt = MeatDataset(train_trans, train_path[tri], target[tri]) # 데이터셋에서 학습데이터 저장
  train_dl = torch.utils.data.DataLoader(train_dt, batch_size = batch_size, shuffle = True) # 데이터셋 기반 배치사이즈+랜덤화를 통해 데이터로더 저장 > 학습데이터는 랜덤화 실행o

  #검증데이터
  valid_dt = MeatDataset(test_trans, train_path[vai], target[vai]) # 데이터셋에서 정답데이터 저장
  valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size = batch_size, shuffle = False) # 데이터셋 기반 배치사이즈+랜덤화를 통해 데이터로더 저장 > 정답데이터는 랜덤화 실행x

  best_score = 0
  patience = 0 # 개선 초기값
  for epoch in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device) # 학습데이터 loop실행
    valid_loss, pred = test_loop(valid_dl, model, loss_fn, device) # 테스트데이터 loop실행, optimizers 제외
    pred = np.argmax(pred, axis = 1) # 예측값 중 행방향 최대값의 인덱스 저장
    # pred = (pred > 0.5).astype(int) # 이진 분류일 때 사용
    score = accuracy_score(target[vai], pred) # 평가지표 > 정답데이터, 예측값

    # print(train_loss, valid_loss, score)
    patience += 1 # 개선한도 증가

    if score > best_score: # best_score 새로 갱신될 경우, 점수 최신화
      patience = 0 # 개선한도 0 초기화
      best_score = score
      torch.save(model.state_dict(), f'model_{i}.pt') # 신경망에서 실행한 결과 파일로 저장하기

    if patience == 5: # 5회이상 개선이 없을 경우 종료
      break
  print(f'Fold-{i}, Best_ACC : {best_score}')
  score_list.append(best_score)
  if is_holdout: # holdout 설정
    break

Fold-0, Best_ACC : 0.964
Fold-1, Best_ACC : 0.8514056224899599
Fold-2, Best_ACC : 0.9879518072289156
Fold-3, Best_ACC : 0.9799196787148594
Fold-4, Best_ACC : 0.9397590361445783


In [22]:
np.mean(score_list)

0.9446072289156626

# 테스트 데이터 추론하기

In [23]:
test_dt = MeatDataset(test_trans, test_path) # 데이터셋에서 test데이터 실행하기
test_dl = torch.utils.data.DataLoader(test_dt, batch_size = batch_size, shuffle = False)

In [24]:
pred_list = []
for i in range(n_splits):
  model = Net(model_name, 3).to(device)
  model_params = torch.load(f'model_{i}.pt', weights_only = True) # 파일에 저장된 결과 중 가중치만 불러오기
  model.load_state_dict(model_params) # 가중치데이터 불러오기

  _, pred = test_loop(test_dl, model, loss_fn, device) # 테스트loop 실행한 결과> 예측값만 저장, epoch_loss 저장x
  pred_list.append(pred)

pred = np.mean(pred_list, axis = 0) # model1~5까지의 평균
pred = np.argmax(pred, axis = 1) # 다중분류문제이므로 클래스번호 필요 > 0, 1, 2 > 가장 큰 값의 인덱스 저장
pred

array([2, 1, 1, ..., 2, 1, 0])